In [1]:
import roman;
print(roman.toRoman(1))

I


In [2]:
from dataclasses import dataclass
import re


@dataclass  # This decorator helps save time from initializing constructor
class Flash_card:
    """This contains a question, its demonstration, possible choices, and correct answers out of the choices"""

    question: str
    choices: list
    correct_choices: list
    demo: str = None

    # Shuffle all choices
    def shuffle_choices(self):
        np.random.shuffle(self.choices)

In [3]:
from termcolor import colored
import numpy as np
import roman


class Folder:
    """This contains multiple flash cards"""

    my_flash = list()
    correct: int = 0

    def total_score(self):
        """This prints the percentage of correct answers"""
        try:
            print(f"You get {self.correct * 100 / len(self.my_flash):.2f}%")
        except:
            print("You have to have at least one question")

    def learn(self):
        """The user chooses correct answer(s) by typing a, b, c, etc. (lower case) and hit enter"""
        # Shuffle order of questions
        np.random.shuffle(self.my_flash)
        for i, flash in enumerate(self.my_flash, 1):
            print(colored(f"Q{roman.toRoman(i)}. {flash.question}", "blue"))  # Print question
            if flash.demo:
                print(flash.demo.strip())  # Print question demonstration
            # Shuffle choices
            flash.shuffle_choices()
            for index, c in enumerate(flash.choices):
                print(
                    colored(f"{index + 1}. {c.capitalize()}", "magenta")
                )  # Print possible choices
            answers = []
            if len(flash.correct_choices) > 1:
                answer = input(
                    "Choose multiple answers"
                )
                while any(int(a) > len(flash.choices) for a in answer):
                    answer = input('You have an invalid answer, please choose again')
                for a in list(answer):
                    answers.append(flash.choices[int(a) - 1])
            else:
                ans = int(input("Choose one correct answer"))
                while ans > len(flash.choices):
                     ans = int(input('You have an invalid answer, please choose again'))
                answers.append(flash.choices[ans - 1])
            if sorted(answers) == sorted(flash.correct_choices):  # Correct answer(s)
                print("Correct!")
                self.correct += 1
            else:  # Incorrect answer(s)
                print(colored(f"Wrong! Correct choice should be: ", "yellow"), end="")
                print(colored("\n".join(flash.correct_choices), "red"))
            print()
        self.total_score()

In [4]:
def read_a_file(file_path: str, general_choice_pattern: str, correct_choice_pattern: str) -> Folder:
    """This reads a text file and returns a Folder instance"""
    doc = Folder()
    with open(file_path, "r", encoding='utf8') as f:
        q, demo, correct_choices, choices = "", "", [], []
        for line in f:
            correct = re.match(correct_choice_pattern, line)
            regular = re.match(general_choice_pattern, line)
            # print(f'line: {line}')
            # Read multiple-lined questions
            if not regular:
                q += line
                # print(f'not regular, q: {q}')
            else:
                # print(f'regular, neat q: {q}')
                if correct:  # Get correct choices
                    correct_choice = line[correct.end():].strip()
                    correct_choices.append(correct_choice)
                    # Any way, the regular choice is added
                    choices.append(correct_choice)
                    # print(f'correct_choice: {correct_choice}')
                else:  # Get incorrect choices
                    regular_choice = line[regular.end():].strip()
                    # print(f'incorrect_choice: {regular_choice}')
                    choices.append(regular_choice)
            if line in ['\n', '\r\n']:
                # Extract the question
                # print(f'q before split: {q}')
                q = q.split(' ', 1)[-1]
                # print(f'q after split: {q}')
                q = q.replace('\n', '').strip()
                # print(f'q after replace: {q}')
                a_flash_card = Flash_card(
                    question=q,
                    choices=choices,
                    correct_choices=correct_choices,
                )
                doc.my_flash.append(a_flash_card)
                # print(f'question: {q}')
                # print(f'answers: {choices}')
                # print(f'correct answer: {correct_choices}')
                # print()
                q, demo, correct_choices, choices = "", "", [], []
    return doc

In [5]:
def create_a_nicer_file(doc: Folder, filename: str):
    """
    This is to create a nicer Q&A file name from doc a Folder instance
    """
    with open(str(filename), "w", encoding='utf8') as f:
        for i, flash in enumerate(doc.my_flash, 1):
            f.write(f"####Q{i}. {flash.question}\n")  # Write question
            if flash.demo:  # Write demo if it exists
                f.write(flash.demo)
            for choice in flash.choices:
                if choice in flash.correct_choices:
                    f.write(f"- [x] {choice}\n")
                else:
                    f.write(f"- [ ] {choice}\n")
            f.write("\n")

In [6]:
file_name = "5215.txt"
doc = read_a_file(file_name, general_choice_pattern=r'^\s*-\s*\[\s*\w*\s*\]\s*', correct_choice_pattern=r'^\s*-\s*\[\s*\w\s*\]\s*')
create_a_nicer_file(doc, file_name.split(".", 1)[0] + ".txt")
doc.learn()

QI. Check which of the following are associated with Bagging instead of Boosting
1. The features (commonly the columns in a data set) and samples/observations (commonly the rows in a data set) may be resampled. and this can be done with or without replacement.
2. This technique is one of the reasons that some kaggle competitions don’t allow teams to merge during competitions (e.g. team #2 and #3 join together)
3. All estimators are weighted equally.


Choose multiple answers 13


Correct!

QII. Which of the following is not an explicit part of the standard Q-learning equation?
1. The policy function
2. Reward prediction error
3. Temporal discounting
4. A state-action value function
5. A learning rate


Choose one correct answer 1


Correct!

QIII. If 5% of your samples have incorrect labels in your available labelled data, which option is likely best to improve model accuracy?
1. Get more samples (even if they are 1% in error)
2. Derive/predict new features from current features in your data set
3. Add/remove features
4. Change your hyperparameter to avoid overfitting


...

Correct!

QV. In Q learning, you are updating the action value function, but there are two parameters which control the manner in which this updating occurs
1. Regularization strength (lamda)
2. Temporal discounting (gamma)
3. Maximum estimation error (epsilon)
4. Learning rate (alpha)


Choose multiple answers 24


Correct!

QVI. cross-validation will lead to lower accuracies than expected with the full training set because only (K-1)/K % of the data is being used for training (e.g. 4/5ths for K=5). The way to improve this is by increasing K. But what is a problem with increasing K?
1. The separated test set is getting small and may bias results of the cross-validation
2. K models have to be trained which takes more time as k increases
3. The number of samples in the data set may not be perfectly divisible by k


Choose one correct answer 56
You have an invalid answer, please choose again 2


TypeError: '>' not supported between instances of 'str' and 'int'